<a href="https://colab.research.google.com/github/mctrinh/ggColab/blob/main/tf2_quick_start_beginer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow 2 quickstart

Use [Keras](https://www.tensorflow.org/guide/keras/sequential_model) to:

- Load a prebuilt dataset.
- Build a neural network machine learning model that classifies images.
- Train this neural network.
- Evaluate the accuracy of the model.

## Set up TensorFlow

Import TensorFlow into your program.

In [3]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.7.0


## Load a dataset

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the sample data from integers to floating-point numbers:

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) and [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [7]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.16931544,  0.33346194,  0.21784386,  0.35699698,  0.29814017,
        -0.2965319 ,  0.24983247,  0.32767147, -0.50568646, -0.17499092]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to probabilities for each class:

In [8]:
tf.nn.softmax(predictions).numpy()

array([[0.10338105, 0.12182288, 0.10852169, 0.12472399, 0.11759497,
        0.06488226, 0.11204927, 0.12111951, 0.05263703, 0.07326736]],
      dtype=float32)

Define a loss function for training using `losses.SparseCategorialCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [10]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: The loss is zero if the model is sure of the correct class.

This untrained mmodel gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [11]:
loss_fn(y_train[:1], predictions).numpy()

2.735181

Before start tranining, configure and compile the model using Keras `Model.compile`. Set the [`optimizer`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) class to `adam`, set the `loss` to the `loss_fn` function defined earlier, and specify a metric to be evaluated for the model by setting the `metrics` parameter to `accuracy`.

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## Train and evaluate your model

Use the `Model.fit` method to adjust your model parameters and minimize the loss:

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 8s 3ms/step - loss: 0.2962 - accuracy: 0.9134
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1437 - accuracy: 0.9570
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1098 - accuracy: 0.9672
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0891 - accuracy: 0.9730
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0754 - accuracy: 0.9764


The `Model.evaluate` method checks the models performance, usually on a "Validation-set" or "Test-set"

In [14]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0814 - accuracy: 0.9769 - 802ms/epoch - 3ms/step


[0.08135673403739929, 0.9768999814987183]

The image classifier is trained to 97.7% accuracy on this dataset.

To make the model returning a probability, we can wrap the trained model, and attach the softmax to it:

In [15]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [16]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.47997383e-07, 2.95115349e-10, 5.20909055e-07, 5.16817927e-05,
        1.03684518e-11, 1.81964737e-07, 1.52615663e-11, 9.99944806e-01,
        8.73739410e-08, 2.40692793e-06],
       [1.14057562e-10, 1.23704776e-05, 9.99973059e-01, 1.30304006e-05,
        2.19087715e-14, 2.96838351e-07, 1.54784328e-08, 8.09454331e-15,
        1.24925475e-06, 2.60021442e-13],
       [6.00764665e-08, 9.99578297e-01, 2.28202189e-05, 2.83573104e-06,
        2.58119562e-05, 8.23290975e-06, 2.20113975e-06, 1.49557847e-04,
        2.10099344e-04, 1.25913488e-07],
       [9.99652386e-01, 2.89574125e-08, 1.80416555e-05, 2.33691435e-06,
        1.33750905e-06, 6.23205381e-07, 2.43131639e-04, 3.21792650e-05,
        4.90302554e-08, 4.98344707e-05],
       [9.26871166e-07, 8.34580599e-11, 4.84943484e-07, 1.85298141e-08,
        9.98196900e-01, 5.31995212e-08, 1.04759515e-07, 3.45076478e-05,
        6.37685673e-07, 1.76627107e-03]], dtype=float32)>